<a href="https://colab.research.google.com/github/trendinafrica/TReND-CaMinA/blob/main/notebooks/Zambia25/08-09-Tue-Wed-NeuronSpiking/Exercises.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" width="300" src="https://raw.githubusercontent.com/trendinafrica/TReND-CaMinA/main/images/CaMinA_logo.png">

# **TReND-CaMinA 2025: Exercises**

**Content creator:** Artemis Koumoundourou

**Acknowledgements:**

# **1. Synaptic Integration — Single vs. Paired Inputs**

**Background:**
You’re studying how a postsynaptic neuron integrates inputs from two presynaptic partners — Neuron A and Neuron B. You've recorded the neuron's membrane potential (Vm) under three conditions:

- When only Neuron A is active

- When only Neuron B is active

- When both A and B are active together

<br>

**Your tasks:**

- Plot the membrane voltage traces for the three conditions.

- Measure the peak response of the postsynaptic neuron in each condition.

- Compare the response to both A and B with the sum of the individual responses.

- Classify the integration as:

 - Linear (combined ≈ A + B)

 - Sublinear (combined < A + B)

 - Supralinear (combined > A + B)

In [ ]:
# Step 1: Load and Preview the Data

# Step 2: Extract Traces

# Step 3: Detect Trials Based on the Stimulus

# Step 4: Extract and Average the Trials

# Step 5: Plot the averaged traces.

# Step 6: Write a Spike Counter

# Step 7: Apply Spike Counter and Compare


In [ ]:
# @title Click to see solution

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Upload and load data
from google.colab import files
uploaded = files.upload()
df = pd.read_csv('SynapticIntegration.csv')

# Extract columns
time = df["Time (ms)"].values
A = df["A_only"].values
B = df["B_only"].values
AB = df["A_and_B"].values
stim = df["Stimulus (%)"].values

# Detect trials based on stimulus onset/offset
def get_trials(stimulus_trace):
    onsets = np.where((stimulus_trace[:-1] == 0) & (stimulus_trace[1:] > 0))[0]
    offsets = np.where((stimulus_trace[:-1] > 0) & (stimulus_trace[1:] == 0))[0]
    trials = []
    for onset in onsets:
        offset = offsets[offsets > onset]
        if len(offset) > 0:
            trials.append((onset, offset[0]))
    return trials

trials = get_trials(stim)
print(f"Detected {len(trials)} trials.")

# Determine trial duration (in samples)
min_len = min(end - start for start, end in trials)

# Initialize arrays to collect aligned trial data
A_trials = []
B_trials = []
AB_trials = []
t_trials = []

for start, end in trials:
    if end - start >= min_len:
        A_trials.append(A[start:start + min_len])
        B_trials.append(B[start:start + min_len])
        AB_trials.append(AB[start:start + min_len])
        t_trials.append(time[start:start + min_len])

# Convert to arrays and average
A_trials = np.array(A_trials)
B_trials = np.array(B_trials)
AB_trials = np.array(AB_trials)
t_trials = np.array(t_trials)

A_avg = A_trials.mean(axis=0)
B_avg = B_trials.mean(axis=0)
AB_avg = AB_trials.mean(axis=0)
t_avg = t_trials[0]  # all time segments are same length

# Plot averaged traces
plt.figure(figsize=(10, 5))
plt.plot(t_avg, A_avg, label="A only (avg)", color='navy')
plt.plot(t_avg, B_avg, label="B only (avg)", color='darkgreen')
plt.plot(t_avg, AB_avg, label="A + B (avg)", color='crimson')
plt.axhline(0, color='black', linestyle='--', alpha=0.5)
plt.xlabel("Time (ms)")
plt.ylabel("Membrane Voltage (mV)")
plt.title("Synaptic Integration: Averaged Voltage Traces")
plt.legend()
plt.grid(True)
plt.show()

# Spike counter
def count_spikes(trace, threshold=-20):
    is_spiking = False
    count = 0
    for v in trace:
        if not is_spiking and v >= threshold:
            count += 1
            is_spiking = True
        elif v < threshold:
            is_spiking = False
    return count

# Apply spike counter and compare
spikes_A = count_spikes(A_avg)
spikes_B = count_spikes(B_avg)
spikes_AB = count_spikes(AB_avg)
expected_sum = spikes_A + spikes_B

print("\nSpike counts in averaged traces:")
print(f"  A only: {spikes_A}")
print(f"  B only: {spikes_B}")
print(f"  A + B: {spikes_AB}")

if spikes_AB == expected_sum:
  print("  → Linear summation")
elif spikes_AB < expected_sum:
  print("  → Sublinear summation")
else:
  print("  → Supralinear summation")

# **2. Inferring Synaptic Connectivity from Voltage Recordings**

**Background:**
You have simultaneous membrane voltage (Vm) recordings from two neurons (Neuron 1 and Neuron 2) recorded during repeated trials where a stimulus turns ON for 500 ms, then OFF for 500 ms. You want to analyze whether these neurons show functional connectivity — i.e., does one neuron’s firing tend to influence the other?

<br>

**Your tasks:**

- Segment the continuous recording into trials based on the stimulus trace.

- Detect spikes in each neuron’s Vm trace during stimulus ON periods.

- Build spike trains from detected spike times.

- Compute and plot the cross-correlation between the two neurons’ spike trains during stimulus ON.

- Interpret whether the cross-correlation suggests a temporal lead-lag relationship (potential synaptic influence).

In [ ]:
# @title Click to see solution

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Load data
df = pd.read_csv("SynapticConnectivity.csv")

time = df["Time (ms)"].values
vm1 = df["Neuron A"].values
vm2 = df["Neuron B"].values

dt = 0.1  # sampling interval in ms
threshold = -20  # spike detection threshold


def detect_spikes(vm, threshold):
    # Detect upward threshold crossings
    spike_indices = np.where((vm[:-1] < threshold) & (vm[1:] >= threshold))[0] + 1
    return spike_indices * dt  # Convert indices to ms

spike_times1 = detect_spikes(vm1, threshold)
spike_times2 = detect_spikes(vm2, threshold)

print("Neuron 1 spike times (ms):", spike_times1)
print("Neuron 2 spike times (ms):", spike_times2)

# Compute all spike time differences within ±window_ms
window_ms = 50
time_diffs = []

for t1 in spike_times1:
    diffs = spike_times2 - t1
    diffs_in_window = diffs[(diffs >= -window_ms) & (diffs <= window_ms)]
    time_diffs.extend(diffs_in_window)

time_diffs = np.array(time_diffs)

# Plot cross-correlogram
bin_size = 1  # ms
bins = np.arange(-window_ms, window_ms + bin_size, bin_size)

hist, edges = np.histogram(time_diffs, bins=bins)

peak_idx = np.argmax(hist)
peak_lag = (edges[peak_idx] + edges[peak_idx + 1]) / 2

print(f"Max correlation at lag = {peak_lag} ms")

if peak_lag > 0:
    print("→ Neuron 1 tends to fire before Neuron 2 (Neuron 1 leads).")
elif peak_lag < 0:
    print("→ Neuron 2 tends to fire before Neuron 1 (Neuron 2 leads).")
else:
    print("→ Neurons tend to fire synchronously.")

plt.bar(edges[:-1], hist, width=bin_size, align='edge')
plt.axvline(peak_lag, color='r', linestyle='--', label=f'Peak lag = {peak_lag} ms')
plt.xlabel('Lag (ms)')
plt.ylabel('Count')
plt.title('Cross-correlogram (spike time differences)')
plt.legend()
plt.show()


# **3. Comparing Neuronal Tuning Using Vector Similarity**

**Background:** You are given membrane voltage recordings from three neurons, all recorded while the same set of increasing stimulus intensities was presented multiple times.

<br>

**Your Tasks:**
- Detect spikes in the voltage traces for each neuron.

-Count how many spikes each neuron fires at each stimulus level.

- Summarize each neuron's responses as a vector of spike counts — one count per stimulus strength.

- Compare the response patterns between neurons by computing the angle between their response vectors.

The **smaller the angle**, the more similar the neurons’ tuning is across stimulus strengths.

<br>

**Questions:**

Which pair of neurons responds most similarly across stimuli?

What does the angle between their response vectors tell you about their coding?

In [ ]:
# Step 1: Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Step 2: Load the CSV file
# Check the structure of the CSV



In [ ]:
# Step 3: Define parameters


# Step 4: Find stimulus onset times

In [ ]:
# Step 5: Define function to detect spikes


# Step 6: Loop through each neuron and count spikes for each stimulus

In [ ]:
# Step 7: Convert spike counts to vectors


In [ ]:
# Step 8: Define function to compute angle between two vectors


# Step 9: Compare neuron pairs

In [ ]:
# @title Click to see solution
# Step 1: Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Step 2: Load the CSV file
# Check for the structure of the CSV
from google.colab import files
uploaded = files.upload()
data = pd.read_csv('NeuronSimilarity.csv')
data.head()

# Step 3: Define parameters
sampling_interval = 0.1  # ms
stim_values = [0, 10, 20, 30, 40]
stim_duration_ms = 250
pre_stim_ms = 50
post_stim_ms = 50

pre_samples = int(pre_stim_ms / sampling_interval)
stim_samples = int(stim_duration_ms / sampling_interval)
post_samples = int(post_stim_ms / sampling_interval)

# Step 4: Find stimulus onset times
stim_onsets = data.index[(data['Stimulus'].shift(1) == 0) & (data['Stimulus'] > 0)].tolist()

# Step 5: Define function to detect spikes
def detect_spikes(trace, threshold=-20):
    return np.where((trace[1:] > threshold) & (trace[:-1] <= threshold))[0]

# Step 6: Loop through each neuron and count spikes for each stimulus
neuron_columns = ['Neuron1', 'Neuron2', 'Neuron3']
spike_counts = {neuron: [] for neuron in neuron_columns}

for neuron in neuron_columns:
    for onset in stim_onsets:
        start = onset
        end = onset + stim_samples
        if end < len(data):
            trace = data[neuron].iloc[start:end].values
            spikes = detect_spikes(trace)
            spike_counts[neuron].append(len(spikes))

# Step 7: Convert spike counts to vectors
vectors = {k: np.array(v) for k, v in spike_counts.items()}

# Step 8: Define function to compute angle between two vectors
def angle_between(v1, v2):
    v1 = np.array(v1)
    v2 = np.array(v2)
    cos_theta = np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))
    return np.degrees(np.arccos(np.clip(cos_theta, -1.0, 1.0)))

# Step 9: Compare neuron pairs
for a in neuron_columns:
    for b in neuron_columns:
        if a < b:
            angle = angle_between(vectors[a], vectors[b])
            print(f"Angle between {a} and {b}: {angle:.2f}°")

# Optional: Plot tuning curves
plt.figure(figsize=(8, 5))
for neuron in neuron_columns:
    plt.plot(stim_values, spike_counts[neuron], marker='o', label=neuron)
plt.xlabel("Stimulus Strength")
plt.ylabel("Spike Count")
plt.title("Tuning Curves")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()
